In [1]:
import pandas as pd
import torch
import os
#check the directory prixfixe for other files like utils, dataprocessor etc
from prixfixe.autosome import AutosomeDataProcessor, AutosomeFirstLayersBlock, AutosomeCoreBlock, AutosomeFinalLayersBlock, AutosomeTrainer, AutosomePredictor
from prixfixe.bhi import BHIFirstLayersBlock,BHICoreBlock
from prixfixe.unlockdna import UnlockDNACoreBlock
from prixfixe.prixfixe import PrixFixeNet
# Load your new test dataframe (adjust the path as necessary)

test_df = pd.read_csv('liba_fli1.csv') #mpra dataset for fli1 sequences

In [2]:
# Remove the first 12 characters from the 'Seq' column
test_df['Seq'] = test_df['Seq'].str[12:]

# Display the updated DataFrame to verify the change
print(test_df['Seq'].head())

0    actagaggatagatctgtgggcatatgcgcgccgtggaccttcggc...
1    actagcacgtacaaatatacaaccatcgaccggttaattatccttg...
2    actcaagccgccacagacccgagggctcctagcacacacatagtaa...
3    acttcctttacacaacgatctaacgagagcgctcacggcttatgtc...
4    actgtggatattagactgaaaccagatttcgcgaaagcaaaccaag...
Name: Seq, dtype: object


In [3]:
# Create a new DataFrame with only the rows where 'clusterID' is 'State_7M'
test_df = test_df[test_df['clusterID'] == 'State_7M'] #since the chip-seq data here was from CMP, I chose state 7M. Testing on the same sequences from other cell states would have been redundant.

# Display the first few rows to verify the filtering
print(test_df.head())

     clusterID           CRS    TF Library  \
1068  State_7M  LibA.Seq1082  Fli1    LibA   
1069  State_7M  LibA.Seq1083  Fli1    LibA   
1070  State_7M  LibA.Seq1085  Fli1    LibA   
1071  State_7M  LibA.Seq1095  Fli1    LibA   
1072  State_7M  LibA.Seq1097  Fli1    LibA   

                                                    Seq  nrepeats  \
1068  actagcacgtacaaatatacaaccatcgaccggttaattatccttg...         1   
1069  actcaagccgccacagacccgagggctcctagcacacacatagtaa...         1   
1070  actgtggatattagactgaaaccagatttcgcgaaagcaaaccaag...         1   
1071  acttacttatcgctccgatgtgaaccaggtctgccctaccgcggtt...         1   
1072  actatgctatccaacaggtgacttctagtcagtggggggttcgctc...         1   

      affinitynum orientation  spacer  mean.norm.raw  mean.norm.adj  \
1068         1.00         fwd      10      -0.359004      -0.135383   
1069         1.00         fwd      20       0.158773       0.382394   
1070         1.00         rev      10      -0.281316      -0.057694   
1071         0.75       

In [4]:
#this is everything that needs to be defined AND make the predictions. The training of the model is in training.ipynb ; since this is a new dataset, 
#some variables like model arhcitecture need to be defined again

import pandas as pd
import torch
import numpy as np
from tqdm import tqdm

# Model parameters that were saved in directory model_weight after training, sequence size and  cuda device id in order to use the gpu
MODEL_LOG_DIR = "model_weight"
#(change it depending on your data seq lnegth from chip and adjust mpra seq accordingly)
SEQ_SIZE = 250
CUDA_DEVICE_ID = 0

# Set up device: use CUDA if available; otherwise, CPU. There werent too many sequences so i have used the cpu below. can be changed
device = torch.device(f"cuda:{CUDA_DEVICE_ID}" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Import model building blocks from files in directory /prixfixe
from prixfixe.autosome import AutosomeCoreBlock, AutosomeFinalLayersBlock
from prixfixe.bhi import BHIFirstLayersBlock
from prixfixe.prixfixe import PrixFixeNet

# Build the model architecture, this is defined also in training.ipynb and needs to be the same. You may use 3 architectures as is or create your own. 
#There is an ipynb here explaining how you can do that https://github.com/de-Boer-Lab/random-promoter-dream-challenge-2022/blob/main/Test_Your_NN_In_Prix_Fixe.ipynb

first = BHIFirstLayersBlock(
    in_channels=5,
    out_channels=320,
    seqsize=SEQ_SIZE,
    kernel_sizes=[9, 15],
    pool_size=1,
    dropout=0.2
)

core = AutosomeCoreBlock(
    in_channels=first.out_channels,
    out_channels=64,
    seqsize=first.infer_outseqsize()
)

final = AutosomeFinalLayersBlock(in_channels=core.out_channels)

generator = torch.Generator()
generator.manual_seed(42)

model = PrixFixeNet(
    first=first,
    core=core,
    final=final,
    generator=generator
)

from torchinfo import summary
print(summary(model, (1, 5, SEQ_SIZE)))

# Load the pre-trained model weights with map_location to ensure compatibility with CPU if needed
model_weights_path = f"{MODEL_LOG_DIR}/model_best_MSE.pth"
model.load_state_dict(torch.load(model_weights_path, map_location=device))
model.to(device)
model.eval()



# Add a 'rev' column if it doesn't exist (assume all sequences are forward)
if 'rev' not in test_df.columns:
    test_df['rev'] = 0

# One-hot encoding function for nucleotides
def one_hot_encode(seq):
    mapping = {
        'A': [1, 0, 0, 0],
        'G': [0, 1, 0, 0],
        'C': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'a': [1, 0, 0, 0],
        'g': [0, 1, 0, 0],
        'c': [0, 0, 1, 0],
        't': [0, 0, 0, 1],
        'N': [0, 0, 0, 0],
        'n': [0, 0, 0, 0]
    }
    return [mapping[base] for base in seq]

# One-hot encode sequences from the 'seq' column containing the sequence which should now be 250 and append the 'rev' value
encoded_seqs = []
Y_test_dev = []
Y_test_hk = []

for i, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    seq_str = row['Seq']
    encoded_seq = one_hot_encode(seq_str)
    # Append the 'rev' value to each one-hot encoded base
    rev_value = [row['rev']] * len(encoded_seq)
    encoded_seq_with_rev = [list(encoded_base) + [rev] for encoded_base, rev in zip(encoded_seq, rev_value)]
    encoded_seqs.append(encoded_seq_with_rev)
   
    
    # Optionally, collect true labels if available
    if 'Dev_log2_enrichment' in test_df.columns:
        Y_test_dev.append(row['Dev_log2_enrichment'])
    if 'Hk_log2_enrichment' in test_df.columns:
        Y_test_hk.append(row['Hk_log2_enrichment'])

# Make predictions for each sequence
pred_expr_dev = []
pred_expr_hk = []

for seq in tqdm(encoded_seqs):
    # Reshape to (1, SEQ_SIZE, 5) and then transpose to (1, 5, SEQ_SIZE)
    seq_array = np.array(seq).reshape(1, SEQ_SIZE, 5).transpose(0, 2, 1)
    seq_tensor = torch.tensor(seq_array, device=device, dtype=torch.float32)
    pred = model(seq_tensor)
    # Assuming pred[0] is Dev enrichment and pred[1] is Hk enrichment
    pred_expr_dev.append(pred[0].detach().cpu().flatten().tolist())
    pred_expr_hk.append(pred[1].detach().cpu().flatten().tolist())

# Optionally, add predictions to your dataframe and save to file
test_df['pred_dev'] = pred_expr_dev
test_df['pred_hk'] = pred_expr_hk
#test_df.to_csv("test_predictions.tsv", sep='\t', index=False)


Using device: cpu


/tmp/11767354/ipykernel_2458540/512105882.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_weights_path, map_location=device))


Layer (type:depth-idx)                        Output Shape              Param #
PrixFixeNet                                   [1, 1]                    --
├─BHIFirstLayersBlock: 1-1                    --                        --
│    └─ModuleList: 2-1                        --                        --
│    │    └─ConvBlock: 3-1                    [1, 160, 250]             7,360
│    │    └─ConvBlock: 3-2                    [1, 160, 250]             12,160
├─AutosomeCoreBlock: 1-2                      --                        --
│    └─ModuleDict: 2-2                        --                        --
│    │    └─Sequential: 3-3                   [1, 320, 250]             420,048
│    │    └─Sequential: 3-4                   [1, 128, 250]             573,696
│    │    └─Sequential: 3-5                   [1, 128, 250]             173,856
│    │    └─Sequential: 3-6                   [1, 128, 250]             229,632
│    │    └─Sequential: 3-7                   [1, 128, 250]         

100%|██████████| 105/105 [00:02<00:00, 47.25it/s]
